## Depth First Search and Strongly Connected Components

In [ ]:
# Install dependencies
!pip install -r requirements.txt --quiet

In [ ]:
import otter
assert (
    otter.__version__ >= "4.4.1"
), "Please reinstall the requirements and restart your kernel."
import networkx as nx
import typing
import numpy as np
from tqdm import tqdm
import pickle
grader = otter.Notebook("dfs_scc.ipynb")

rng_seed = 42

In [ ]:
# Load test cases
file_path = "generated_testcases.pkl"

# Load the variables from the pickle file
with open(file_path, "rb") as file:
    loaded_data = pickle.load(file)
file.close()
inputs, outputs = loaded_data

#### Representing graphs in code

There are multiple ways to represent graphs in code. In class we covered [adjacency matrices](https://people.eecs.berkeley.edu/~vazirani/algorithms/chap3.pdf#page=2) and [adjacency lists](https://people.eecs.berkeley.edu/~vazirani/algorithms/chap3.pdf#page=3). There is also the edge list representation, in which you store the edges in a single 1 dimensional list. In general for CS170 and in most cases, we choose to use the adjacency list representation since it allows us to efficiently search over a node's neighbors.

In many programming problems, verticies are typically labelled $0$ through $n-1$ for convenience (recall that arrays and lists in most languages begin at index 0). This allows us to represent an adjacency list using a list of lists that store ints. Given an edge list, the following code will create an adjacency list for an **unweighted directed graph**.

In [ ]:
def generate_adj_list(n, edge_list):
    """
    args:
        n:int = number of nodes in the graph. The nodes are labelled with integers 0 through n-1
        edge_list:List[Tuple(int,int)] = edge list where each tuple (u,v) represents the directed 
            edge (u,v) in the graph
    return:
        A List[List[int]] representing the adjacency list 
    """
    adj_list = [[] for i in range(n)] 
    for u, v in edge_list:
        adj_list[u].append(v)
    for nodes in adj_list:
        nodes.sort()
    return adj_list

### Q1) Depth First Search
In this question, you will have to implement a DFS traversal for the given adj_list. As you traverse the graph, you should update a prev array, where prev[v] = u if you came to v from u. Return this list prev. By default prev[start] should equal start. 

**For all subparts, your DFS should prioritize visiting lower valued vertices first.**

> **Task 1.1**
>* Given a graph represented as an adjacency list, perform a DFS traversal that prioritizes visiting the lower valued nodes first
>* Return a list of length=number of nodes where the ith element in this list contains the node that explored node i. If node i is the root of the dfs tree, then the ith element should be node i.
>* Make sure to traverse every node present in the graph, even if the graph is disconnected.

_Points:_ 2

In [ ]:
def dfs(adj_list):
    """
    args:
        adj_list:List[List[int]] = the adjacency list that represents our input graph
    return:
        A List[int] where the ith element represents the node that called dfs(i). If a 
            DFS is started from a vertex, then List[i] = i.
    """
    ...

In [ ]:
grader.check("q1.1")

#### Pre and Postorder Values

In class we showed how to use DFS to check if there exists a path between two nodes, topologically sort nodes, and find SCC's. In those algorithms, pre and post numbers were crucial.

>**Task 1.2**
>* Rework your implementation of DFS in this next cell to allow it to generate pre and post order numbers for each node.
>* Your smallest preorder number should be 1. Your largest postorder number should be  $ 2 \times \text{(number of vertices)}$.
>* Return two lists of tuples. Each list should contain a tuple of two values Tuple(Node, Time Visited). The first list should contain the tuples with the preorder visits, and the second list should contain the tuples with the postorder visits.
>* We have provided some boiler plate code to get you started. Feel free to write your own if you find that easier.
>* Feel free to refer to Figure 3.6(b) in section 3.2.3 of the DPV notes for how your pre and post order values should look like.
>>* If the input was the left most component, you should return [(A, 1), (B, 2), (E, 4), (I, 5), (J, 6)], [(B, 3), (J, 7), (I, 8), (E, 9), (A, 10)].
>>* Both lists should be sorted according to the second element in the tuple.

_Points:_ 2

In [ ]:
def get_pre_post(adj_list):
    """
    args:
        adj_list:List[List[int]] = the adjacency list that represents our input graph
    return:
        List[Tuple(int, int)], List[Tuple(int, int)] representing the pre and post order values
            respectively. Each tuple should have a vertex as its first entry, and the pre/post order
            value as its second entry.
    """
    time = 1
    pre = []
    post = []
    ...
    return pre, post

In [ ]:
grader.check("q1.2")

#### Tree, Forward, Back, Cross Edges

As we perform DFS traversals and create DFS trees and DFS forests within our graph, we would like to classify our edges according to how they appear in the resulting DFS forest. These classifications can provide us with insights about our graph. For example, the presence of a back edge (u, v) tells us that we have a cycle within this graph that includes all the tree edges on the path from v to u and the back edge (u, v).

>**Task 1.3**
>* Given the adjacency list of a graph, add each edge present in the edge set to the correct classification according the DFS traversal you implemented in part 1.
>* Don't modify the initialization of the edges_lookup dictionary.

_Points:_ 2

In [ ]:
def categorize_edges(adj_list):
    """
    args:
        adj_list:List[List[int]] = the adjacency list that represents our input graph
    return:
        Dictionary({
            'tree': set(),
            'forward': set(),
            'cross': set(),
            'back': set()  
        }) where each set() contains the edges that belong to the corresponding edge type
    """
    edges_lookup = {
        'tree': set(),
        'forward': set(),
        'cross': set(),
        'back': set()
    }
    ...
    return edges_lookup

In [ ]:
grader.check("q1.3")

### Q2) Strongly Connected Components

We will now see how we can tie the concepts of DFS traversals and pre/post order values to obtain the strongly conencted components within a graph. SCC meta graphs are useful in that they allow us to construct DAGs, linearize a graph that contains cycles and obtain equivalence classes within a graph among other use cases.

S. Rao Kosaraju solved this problem of finding the SCCs within a graph in 1978 with the Kosaraju-Sharir Algorithm (which is the algorithm presented in the DPV notes section 3.4.2). The first step of this algorithm is to reverse the graph.

> **Task 2.1**
>* Given a graph represented as an adjacency list, return the acjacency list that represents the reversed graph (the vertex set is the same as the input graph, the edge set contains (v, u) if and only if (u, v) is present in the input graph's edge set).

_Points:_ 2

In [ ]:
def reverse_graph(adj_list):
    """
    args:
        adj_list:List[List[int]] = the adjacency list that represents our input graph
    return:
        List[List[int]] representing the adjacency list of the reversed input graph
    """
    reversed_adj_list = [[] for i in range(len(adj_list))]
    ...
    return reversed_adj_list

In [ ]:
grader.check("q2.1")

Now, you get to complete the rest of the algorithm to find the SCCs within a graph.

> **Task 2.2**
>* Given a graph represented as an adjacency list, return a list of SCC components.
>* The SCC components should be represented as sets that contain only the nodes present in that SCC.

_Points:_ 2

In [ ]:
def find_SCCs(adj_list):
    """
    args:
        adj_list:List[List[int]] = the adjacency list that represents our input graph
    return:
        List(Set(int, ...), ...) a list of sets where each set contains all the nodes 
            that belong to the corresponding SCC
    """
    scc_list = []
    ...
    return scc_list

In [ ]:
grader.check("q2.2")

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit.

In [ ]:
grader.export(pdf=False, force_save=True, run_tests=True)